### Task 2: Determine within which grid cell a point/particle falls into
### Task 3: Interpolate velocity to the particle lat/lon 
### Task 4: Advance forward

- variables u,v (current velocity components)
- use only surface data: depth (z) = -2.0 
- use first time for now [0]

Note: current velocities u, v are on different grids, u(eta_u,xi_u) and v(eta_v, xi_v).
Latitudes and longitudes corresponding values for current are in the coordinate arrayes lat_u, lon_u, lat_v, lon_v


In [5]:
# task1: acquire data
import numpy as np
import pandas as pd
import xarray as xr

# url='https://oceanmodeling.ucsc.edu:8443/thredds/dodsC/ccsra_2016a_phys_agg_zlevs/fmrc/CCSRA_2016a_Phys_ROMS_z-level_(depth)_Aggregation_best.ncd'
url='dataset_2010.nc'
actual = xr.open_dataset(url)

# ds = actual.copy()              # ("ds.nc")


lat1 = 38.0
lon1 = -123.0

In [ ]:
"""
Chunks down and downloads the remote dataset from UCSC's server
"""
# end_index = len(actual.time.values)
# master = actual.sel(time = actual.time.values[0])
# for i in range(1, end_index):
#     temp = actual.sel(
#         time = actual.time.values[i],
#         z = slice(0, 1)
#     )
#     master = xr.concat([master, temp], dim='time')
#     if i % 100 == 0:
#         name = f"backup_{i}.nc"
#         master.to_netcdf(name)
#     print(i)
# master.to_netcdf("ds.nc")
# t = xr.open_dataset("ds.nc")
# t

In [ ]:
# Task2: find nearest grid points for current components u and v.

for x in range(len(ds.lon_u)):
    print(ds.lon_u[0, x].lon_u.values)

In [8]:
# Task3: interpolate velocity (u,v) to lat1 and lat2 (this point might not fall on a grid point)

actual = actual.interp(eta_u=actual.eta_rho, xi_u=actual.xi_rho, method='linear')
actual = actual.interp(eta_v=actual.eta_rho, xi_v=actual.xi_rho, method='linear')

In [9]:
actual

<xarray.Dataset>
Dimensions:      (z: 11, eta_rho: 181, xi_rho: 186, ocean_time: 1, time: 101)
Coordinates:
  * z            (z) float64 -250.0 -200.0 -150.0 -100.0 ... -10.0 -5.0 -2.0
    lat_rho      (eta_rho, xi_rho) float64 30.0 30.0 30.0 ... 48.0 48.0 48.0
    lat_u        (eta_rho, xi_rho) float64 30.0 30.0 30.0 30.0 ... 48.0 48.0 nan
    lat_v        (eta_rho, xi_rho) float64 30.05 30.05 30.05 ... nan nan nan
    lon_rho      (eta_rho, xi_rho) float64 -134.0 -133.9 ... -115.6 -115.5
    lon_u        (eta_rho, xi_rho) float64 -133.9 -133.9 -133.8 ... -115.5 nan
    lon_v        (eta_rho, xi_rho) float64 -134.0 -133.9 -133.8 ... nan nan nan
  * ocean_time   (ocean_time) datetime64[ns] 2022-07-09
  * time         (time) datetime64[ns] 2011-01-02 2011-01-03 ... 2011-04-12
    time_run     (time) datetime64[ns] 2011-01-02 2011-01-03 ... 2011-04-12
Dimensions without coordinates: eta_rho, xi_rho
Data variables: (12/19)
    angle        (time, eta_rho, xi_rho) float64 0.0004363 ... 0.0006485
    h            (time, eta_rho, xi_rho) float64 4.497e+03 4.497e+03 ... 10.0
    mask_rho     (time, eta_rho, xi_rho) float64 1.0 1.0 1.0 1.0 ... 0.0 0.0 0.0
    mask_u       (time, eta_rho, xi_rho) float64 1.0 1.0 1.0 1.0 ... 0.0 0.0 nan
    mask_v       (time, eta_rho, xi_rho) float64 1.0 1.0 1.0 1.0 ... nan nan nan
    ubar         (time, eta_rho, xi_rho) float64 0.01585 0.01274 ... nan nan
    ...           ...
    w            (time, z, eta_rho, xi_rho) float32 -0.001627 -0.001627 ... nan
    time_offset  (time) datetime64[ns] 2011-01-02 2011-01-02 ... 2011-01-02
    eta_u        (eta_rho) int64 0 1 2 3 4 5 6 7 ... 174 175 176 177 178 179 180
    xi_u         (xi_rho) int64 0 1 2 3 4 5 6 7 ... 179 180 181 182 183 184 185
    eta_v        (eta_rho) int64 0 1 2 3 4 5 6 7 ... 174 175 176 177 178 179 180
    xi_v         (xi_rho) int64 0 1 2 3 4 5 6 7 ... 179 180 181 182 183 184 185
Attributes: (12/46)
    file:                            output/da_fwd_002.nc
    format:                          netCDF-3 64bit offset file
    Conventions:                     CF-1.4
    type:                            ROMS/TOMS history file
    title:                           UCSC California Current System, wc12.0_c...
    state_vector:                    zeta, ubar, vbar, u, v, temp, salt
    ...                              ...
    NCO:                             "4.5.2"
    _CoordSysBuilder:                ucar.nc2.dataset.conv.CF1Convention
    cdm_data_type:                   GRID
    featureType:                     GRID
    location:                        Proto fmrc:CCSRA_2016a_Phys_ROMS_z-level...
    DODS_EXTRA.Unlimited_Dimension:  ocean_time

In [ ]:
# Task4: move particle forward 
# we need to calculate the position of the particle if it moves for one time step (a day in this data set) with velocity u, v
# there are different methods to implement that, we will use the Runge-Kutta order 4. but you can start w an smipl

class point:
    def __init__(self, x, y, time):
        self.x = x
        self.y = y
        self.time = time
    def move(self, u, v, dt):
        # TODO: Do the thing
        self.x += u * dt
        self.y += v * dt
    def __str__(self):
        return f"({self.x}, {self.y})"